In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
%matplotlib inline
import time
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid
from os import listdir, makedirs, getcwd, remove
from os.path import isfile, join, abspath, exists, isdir, expanduser
from PIL import Image
from torch import nn, optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
import torchvision
import torch
from torchvision import transforms, datasets, models
from collections import OrderedDict

In [ ]:
!ls ../input/resnet152/

In [ ]:
cache_dir = expanduser(join('~', '.torch'))
if not exists(cache_dir):
    makedirs(cache_dir)
models_dir = join(cache_dir, 'models')
if not exists(models_dir):
    makedirs(models_dir)

In [ ]:
!cp ../input/vgg19-model///* ~/.torch/models/

In [ ]:
!ls ~/.torch/models

In [ ]:
#mv ~/.torch/models/resnet152.pth ~/.torch/models/resnet152-b121ed2d.pth

In [ ]:
!ls ../input/pytorch-challange-flower-dataset/flower_data/

In [ ]:
data_dir = '../input/pytorch-challange-flower-dataset/flower_data/flower_data'
train_dir = data_dir + '/train'
valid_dir = data_dir + '/valid'

In [ ]:
train_transforms = transforms.Compose([transforms.RandomRotation(30),                                       
                                transforms.CenterCrop(224),
                                transforms.RandomHorizontalFlip(),       
                                transforms.ToTensor(),
                                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

val_transforms = transforms.Compose([transforms.CenterCrop(224),
                                transforms.ToTensor(),
                                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

# TODO: Load the datasets with ImageFolder
train_datasets = datasets.ImageFolder(train_dir, transform=train_transforms)
val_datasets = datasets.ImageFolder(valid_dir, transform=val_transforms)
# TODO: Using the image datasets and the trainforms, define the dataloaders
train_loader = torch.utils.data.DataLoader(train_datasets, batch_size=128)
val_loader = torch.utils.data.DataLoader(val_datasets, batch_size=128)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
model = models.vgg19(pretrained=True)

In [ ]:
for param in model.parameters():
    param.requires_grad = False
    
model.classifier = nn.Sequential(nn.Linear(25088, 4096),
                                 nn.ReLU(),
                                 nn.Dropout(),
                                 nn.Linear(4096, 4096),
                                 nn.ReLU(),
                                 nn.Dropout(),
                                 nn.Linear(4096, 102),
                                 nn.LogSoftmax(dim=1))

criterion = nn.NLLLoss()
# Probar con 0.001
optimizer = optim.Adam(model.classifier.parameters(), lr=0.00003)
model.to(device)

In [ ]:
epochs = 50
steps = 0
train_losses, val_losses, val_accuracy = [], [], []
val_loss_min = np.Inf
print_every = 5
for epoch in range(epochs):
    running_loss = 0
    model.train()
    for inputs, labels in train_loader:
        steps += 1

        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        logps = model(inputs)
        loss = criterion(logps, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    else:
        val_loss = 0
        accuracy = 0
            
        with torch.no_grad():
            model.eval()
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                logps = model(inputs)
                val_loss += criterion(logps, labels)

                ps = torch.exp(logps)
                top_p, top_class = ps.topk(1, dim=1)
                equals = top_class == labels.view(*top_class.shape)
                accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
        
        train_losses.append(running_loss/len(train_loader))
        val_losses.append(val_loss/len(val_loader))
        val_accuracy.append(accuracy/len(val_loader))
        print('Epoch {}/{}'.format(epoch + 1, epochs))
        print('Train loss: {}'.format(running_loss/print_every))
        print('Val loss: {}'.format(val_loss/len(val_loader)))
        print('Val accuracy: {}'.format(accuracy/len(val_loader)))
        
        if val_loss <= val_loss_min:
            print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
            val_loss_min,
            val_loss))
            torch.save(model.state_dict(), 'model.pt')
            val_loss_min = val_loss

In [ ]:
plt.plot(train_losses, label="Train loss")
plt.plot(val_losses, label="Validation loss")
plt.legend()

In [ ]:
plt.plot(val_accuracy, label="Validation Accuracy")

In [ ]:
torch.save(model.state_dict, "../input/model.pwf")

In [ ]:
ls